In [ ]:
from services.notebook.model_loader import ModelLoader
import services.adapters.sqlite_adapter as sa
from services.discovery.dataset_discovery import EventDatasetDiscovery
from tests.samples.sources import SIMPLE_BIG_DATA, SIMPLE_CSV
from datetime import datetime
import services.notebook.visualization as vis
import services.adapters.adapter_factory as af

adapter = af.get_or_create_adapter(SIMPLE_BIG_DATA)
discovery = EventDatasetDiscovery(
    SIMPLE_BIG_DATA, adapter, datetime(2021, 1, 1), datetime(2022, 1, 1)
)
dd = discovery.discover_dataset()
ml = ModelLoader()
m = ml.create_dataset_model(dd)
m._to_globals(globals())

In [ ]:
(app_install.country_code.is_not_null).segmentation(start_dt="2021-12-01",end_da,time_group="day")